#### Import the variables and definitions from the accompanying music_def.py file 

In [20]:
from music_def import *

***

### Define your key, scale, and octave

In [ ]:
# First choice with regards to music
display(vert_note)
display(vert_scale)
display(vert_octave)


In [ ]:
note_index = notes_to_index[def_note.value]
scale_num = scale_index[def_scale.value]
scale_clean = all_scales[scale_num]

print("--------------------------------------------------------------------------------------------------------------------------")

print("You have chosen the Key,", def_note.value, "which corresponds to the index: \n", note_index)
print("The chosen scale is a ", def_scale.value, "which corresponds to the midi indices: \n", scale_clean)
print("The octave that you have chose to map your notes onto is: \n ", def_octave.value)

print("If this is incorrect, please check with the drop down menu above and run this cell again.")

# Mapping chosen scale onto each other 
scale = [] 
for ix in scale_clean:
    a = ix + (len_oct*def_octave.value) + note_index
    scale.append(a)

print("--------------------------------------------------------------------------------------------------------------------------")

***

### Looking at datafile

##### The variable "file" is for the whole data file
##### The variable "data" is used for the part that we want to convert to music 

In [ ]:
# For now we can take the noisy C60 IR as the guinea pig again 
# This part will differ depending on your data 

file = np.loadtxt('C60plus.csv',skiprows=1, delimiter=',')
data = file[:,1]

# First look at the data 
plt.plot(file[:,0],file[:,1])

### If I want to zoom in or out?
##### Note: the more you zoom the more features your resulting sound will have depending on the noise of your data

In [ ]:
# Selecting Data range 
# Zoom widget, this cannot be predefined as it must be redefined after data gets assigned to your data 
zoom_range = widgets.FloatRangeSlider(
    min=np.min(data),
    max=np.max(data),
    value=[np.min(data), np.max(data)],
    step=0.01,
    description='Zoom Range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
display(zoom_range)

In [ ]:
# Look at data again with zoomed values on y axis 

low = zoom_range.value[0]
high = zoom_range.value[1]

# Or if you wish to manually define the low and high points 
# low = 
# high =

# Plotting with the zoom
plt.plot(file[:,0],file[:,1])
plt.ylim(low , high)
plt.show()

# Once you are happy with the range, we bin the data to match our chosen scale

bins = np.linspace(low ,high,len(scale)) #our original data split into 15 bins, corresponding to our 15 notes
abs_binned = np.digitize(data,bins) #our data put into these 15 bins
unique_bins = np.unique(abs_binned) # array of our unique bins 

revalue = dict(zip(unique_bins, scale)) # dictionary to map our scale to the binned values of our IR absorption
notes = [revalue.get(n, n) for n in abs_binned] #put the absorptions onto the scale

****

### Definine rest of parameters for MIDIUtil to work

#### If you are confused on the choices please visit the MIDIUtil Docs

In [ ]:
display(vert_channel)
display(vert_tempo)
display(vert_volume)
print("About time signature")
display(vert_bpb)
display(vert_type_beat)

In [ ]:
# Set up some variables for the MIDIUtil program to work
channel  = int(def_channel.value)
tempo    = int(def_tempo.value)      
volume   = int(def_volume.value)

display(vert_name)
display(vert_int)

In [28]:
# Writing a MIDI file based on your choices 

MyMIDI = MIDIFile(1)  # One track, defaults to format 1 (tempo track is created
                      # automatically)
MyMIDI.addTempo(track, time, tempo)
MyMIDI.addTimeSignature(track, time, def_bpb.value , note_name_dictionary[def_type_beat.value], clocks_per_tick, notes_per_quarter )   #add a time signature, here just 4/4

#loop over each time step 
i=0
for n in notes[::4]:  #loop over each note
    MyMIDI.addNote(track, channel, n, time+i, duration, volume)
    i=i+1

#save the resulting midi file.    
with open(str(def_name.value)+".mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)